In [20]:
import pandas as pd
import numpy as np
import sklearn as sk
from datetime import timedelta
from sklearn import model_selection, tree

In [35]:
filename = 'shot_logs.csv'
data = pd.read_csv(filename)
data = data.dropna()
print(data.shape)
data.head()

(122502, 21)


,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148
5,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,6,2,8:15,9.1,2,...,18.4,2,missed,"Williams, Deron",101114,2.6,0,0,brian roberts,203148


In [26]:
def timeToSeconds(x):
    mins, secs = map(float, x.split(':'))
    td = timedelta(minutes=mins, seconds=secs)
    return td.total_seconds()

In [36]:
# begin to separate the data into features and labels
labels = data['SHOT_RESULT'].map(dict(made=1, missed=0))
feature_data = data.drop(['SHOT_RESULT', 'MATCHUP', 'GAME_ID', 'CLOSEST_DEFENDER','player_name', 'LOCATION','W'], axis=1)
feature_data['GAME_CLOCK'] = feature_data['GAME_CLOCK'].apply(timeToSeconds)

#feature_data['SHOT_CLOCK'].apply(timeToSeconds)

feature_data.head()

,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_id
0,24,1,1,69.0,10.8,2,1.9,7.7,2,101187,1.3,1,2,203148
1,24,2,1,14.0,3.4,0,0.8,28.2,3,202711,6.1,0,0,203148
3,24,4,2,707.0,10.3,2,1.9,17.2,2,203900,3.4,0,0,203148
4,24,5,2,634.0,10.9,2,2.7,3.7,2,201152,1.1,0,0,203148
5,24,6,2,495.0,9.1,2,4.4,18.4,2,101114,2.6,0,0,203148


In [37]:
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(feature_data, labels, test_size=.8, train_size=.2)
dTree = sk.tree.DecisionTreeClassifier(criterion='entropy')
dTree.fit(x_train, y_train)
print(sk.metrics.accuracy_score(y_test, dTree.predict(x_test)))

1.0
